In [1]:
%matplotlib qt

import hyperspy.api as hs

from generalizedplanck.components import (GeneralizedPlanck, Reflectance, 
                                          IdealSqrtAbsorption, UrbachTail)
from generalizedplanck.utils import get_nk
from generalizedplanck.data import gaas_cl
import matplotlib.pyplot as plt

In [2]:
def fit_gen_planck(signal=None, absorption_coeff=None, reflectance=None, tail=None,
                   gui=False, plotComponents=False, fit_lim=None, add_component=None, 
                   **kwargs):
    '''

    Parameters
    ----------
    absorption : TYPE, optional
        DESCRIPTION. The default is None.
    reflectance : TYPE, optional
        DESCRIPTION. The default is None.
    tail : TYPE, optional
        DESCRIPTION. The default is None.
    Eg : TYPE, optional
        DESCRIPTION. The default is None.
    g : TYPE, optional
        DESCRIPTION. The default is None.
    p : TYPE, optional
        DESCRIPTION. The default is None.
    T : TYPE, optional
        DESCRIPTION. The default is None.
    d : TYPE, optional
        DESCRIPTION. The default is None.
    Efv : TYPE, optional
        DESCRIPTION. The default is None.
    Efc : TYPE, optional
        DESCRIPTION. The default is None.
    **kwargs : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    '''
    from matplotlib.pyplot import ion
    #initialize genp with default values for parameters
    genp = GeneralizedPlanck(reflectance=reflectance,
                             ideal_abs_coeff=absorption_coeff,
                             tail=tail
                             )
    
    for param in genp.parameters:
        if param.name in kwargs:
            if len(kwargs[param.name]) == 4:
                param.free = kwargs[param.name][0]
                param.value = kwargs[param.name][1]
                param.bmin = kwargs[param.name][2]
                param.bmax = kwargs[param.name][3]
            elif len(kwargs[param.name]) == 2:
                param.free = kwargs[param.name][0]
                param.value = kwargs[param.name][1]
            else:
                text = ('It accepts only [free, value] or'
                        ' [free, value, bmin, bmax]')
                raise NotImplementedError(text)
    m = signal.create_model()
    m.append(genp)
    m.plot()
    plt.pause(1)
    optimizer = 'lm'
    loss_function = 'ls'
    if 'optimizer' in kwargs:
        optimizer = kwargs['optimizer']
    elif 'loss_function' in kwargs:
        loss_function = kwargs['loss_function']
    if fit_lim:
        m.set_signal_range(*fit_lim)
    if gui:
        m.gui()
    else:
        m.fit(bounded=True, optimizer=optimizer, loss_function=loss_function)
    if plotComponents:
        genp.plotComponents()
    m.print_current_values()
    return m

Create a fake GaAs CL signal

In [3]:
s = gaas_cl()

Creating a reflectance object for the Generalised Planck law

In [4]:
n1 = get_nk(n=1)
n2 = get_nk(shelf='main', book='GaAs', page='Papatryfonos')
gaas_r = Reflectance(theta=0, n1=n1, n2=n2, pol=None) 

Create an ideal sqrt absorptio coefficient plus an Urbach tail

In [5]:
gaas_ideal_abs = IdealSqrtAbsorption(Eg=1.42, E0=1.6, a0=14800)
gaas_tail = UrbachTail(g=0.015)

Put everything in the Generalised Planck component

In [6]:
gaas_genp = GeneralizedPlanck(Eg=1.42, g=0.01, p=0.8, T=300, d=150, 
                              Efv=0, Efc=0.1,
                              reflectance=gaas_r,
                              ideal_abs_coeff=gaas_ideal_abs,
                              tail=gaas_tail)

Fitting example

In [7]:
m = fit_gen_planck(s, absorption_coeff=gaas_ideal_abs, reflectance=gaas_r, tail=gaas_tail,
               plotComponents=True, gui=True,
               Eg=[True,1.39,1.39,1.42],
               g=[True, 0.01, 0.001, 0.05],
               p=[False, 0.89],
               d=[False, 2000],
               T=[False, 295],
               Efv=[True, 0, -0.1, 0.1],
               Efc=[False, -0.5])

Accordion(children=(VBox(children=(Checkbox(value=True, description='active'), HBox(children=(FloatText(value=…

Parameter Name,Free,Value,Std,Min,Max,Linear
Eg,True,1.39,,1.39,1.42,False
g,True,0.01,,0.001,0.05,False
p,False,0.89,,0,1,False
T,False,295,,0,,False
d,False,2000,,0,,False
Efv,True,0,,-0.1,0.1,False
Efc,False,-0.5,,,,False
theta,Twinned,0,,,,False
a0,Twinned,14800,,,,False
E0,Twinned,1.6,,,,False
